- Carga el CSV en Spark.

In [ ]:
from pyspark.sql import SparkSession

# Create or get a Spark session
spark = SparkSession.builder.appName("Read Compressed CSV from S3").getOrCreate()

# Define the path to your S3 bucket and compressed CSV files
s3_bucket_path = "s3://mdge-e3-2024/*.csv.gz"

# Read the compressed CSV files into a DataFrame
df = spark.read.csv(s3_bucket_path, header=True, inferSchema=True)

In [5]:
# Add "year" and "month" columns based on "fecha_registro"
from pyspark.sql import functions as F
df = df.withColumn("year", F.year("fecha_registro"))
df = df.withColumn("month", F.month("fecha_registro"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- Guarda el CSV como parquet en S3, particionalo por `catalogo`. (Utiliza todos los trucos que consideres).

In [ ]:
# Define output directory
output_directory = "s3://mdge-e3-2024/parquet_files_partitioned/"

# Write the DataFrame to Parquet, partitioned by 'categoria', 'year', and 'month'
df.write.partitionBy("categoria", "year", "month").mode('overwrite').parquet(output_directory)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- Carga el parquet en Spark.

In [ ]:
# Read the Parquet files into a DataFrame
df_parquet = spark.read.parquet(output_directory)

Contesta las siguientes preguntas utilizando PySpark. Realiza el siguiente análisis **(por año)** y sobre todos los catálogos.

- ¿Cuántos catálogos diferentes tenemos?

In [ ]:
from pyspark.sql.functions import countDistinct

# Count distinct catalogs
distinct_catalogs = df.agg(countDistinct("catalogo").alias("distinct_catalogs"))
distinct_catalogs.show()

- ¿Cuáles son los20 catálogos con más observaciones? Guarda la salida de este query en tu bucket de S3, lo necesitaremos más adelante.

In [ ]:
from pyspark.sql.functions import desc

# Find top 20 catalogs by number of observations
top_20_catalogs = df.groupBy("catalogo").count().orderBy(desc("count")).limit(20)
top_20_catalogs.show()

# Save the output to S3
top_20_catalogs.write.mode('overwrite').parquet("s3://mdge-e3-2024/top_20_catalogs/")


- ¿Tenemos datos de todos los estados del país? De no ser así, ¿cuáles faltan?

In [ ]:
all_states = [
    "aguascalientes", "baja california", "baja california sur", "campeche", "chiapas",
    "chihuahua", "ciudad de mexico", "coahuila", "colima", "durango", "guanajuato",
    "guerrero", "hidalgo", "jalisco", "mexico", "michoacan", "morelos", "nayarit",
    "nuevo leon", "oaxaca", "puebla", "queretaro", "quintana roo", "san luis potosi",
    "sinaloa", "sonora", "tabasco", "tamaulipas", "tlaxcala", "veracruz", "yucatan", "zacatecas"
]

# Get distinct states from DataFrame
states_in_data = df.select("estado").distinct().rdd.flatMap(lambda x: x).collect()

# Find missing states
missing_states = [state for state in all_states if state not in states_in_data]
print("Missing States:", missing_states)

- ¿Cuántas observaciones tenemos por estado?

In [ ]:
# Count observations by state
observations_by_state = df.groupBy("estado").count()
observations_by_state.show()

- De cada estado obten: el número de catalogos diferentes por año, ¿ha aumentado el número de catálogos con el tiempo?

In [ ]:
from pyspark.sql.functions import year

# Count distinct catalogs per state per year
catalogs_per_state_per_year = df.groupBy("estado", "year").agg(countDistinct("catalogo").alias("distinct_catalogs"))
catalogs_per_state_per_year.show()

# To check if the number of catalogs has increased over the years, we will order the results
trend_catalogs = catalogs_per_state_per_year.orderBy("estado", "year")
trend_catalogs.show()

Utilizando Spark contesta las siguientes preguntas a partir **del catálogo que
le tocó a tu equipo**. Recuerda trabajar en el archivo con los datos particionados
de otra manera tus queries van a tardar mucho.

- ¿Cuańtas marcas diferentes tiene tu categoría?

- ¿Cuál es la marca con mayor precio? ¿En qué estado?

- ¿Cuál es la marca con menor precio en CDMX? (en aquel entonces Distrito Federal)

- ¿Cuál es la marca con mayores observaciones?

- ¿Cuáles son el top 5 de marcas con mayor precio en cada estado? ¿Son diferentes?

- ¿Cuáles son el top 5 de marcas con menor precio en CDMX? (en aquel entonces Distrito Federal)

- ¿Cuáles son el top 5 de marcas con mayores observaciones? ¿Se parecen a las de nivel por estado?

- ¿Ha dejado de existir alguna marca durante los años que tienes? ¿Cuál? ¿Cuándo desapareció?

- Genera una gráfica de serie de tiempo por estado para la marca con mayor precio -en todos los años-, donde el eje equis es el año y el eje ye es el precio máximo.